In [1]:
import pandas as pd
import ppscore as pp
import re

In [2]:
df_forecast = pd.read_csv("forecast_starting_data.csv")
df_training = pd.read_csv("training_data.csv")
df_macro = pd.read_csv("macro_data.csv")

/var/folders/jh/hbpckc3j0vj731hydfqf993h0000gn/T/ipykernel_21558/2315000146.py:2: DtypeWarning: Columns (51) have mixed types. Specify dtype option on import or set low_memory=False.
  df_training = pd.read_csv("training_data.csv")


IN THE NEXT STEP WE WILL FILTER THE MACRO DATA

In [31]:
def format_date(input: str):
    if len(input) == 9:
        return input[-4:] + "0" + input[0]
    elif len(input) == 10:
        return input[-4:] + input[0:2]
    else:
        return "YOU BEEN A VERY BAD BOY TODAY >:("
# TESTING FORMAT_DATE()
print(format_date("12/31/2020"))
print(format_date("2/31/2020"))
print(format_date("2/31/20"))

202012
202002
YOU BEEN A VERY BAD BOY TODAY >:(


In [42]:
import numpy as np
import datetime

In [109]:
macro_final = pd.DataFrame()
for i, row in df_macro.iterrows():
    if re.search(r"2018$", str(row["Mnemonic"])) or re.search(r"2019$", str(row["Mnemonic"])) or re.search(r"2020$", str(row["Mnemonic"])) or re.search(r"2021$", str(row["Mnemonic"])):
        t1 = pd.DataFrame(row).T
        month_code = format_date(t1['Mnemonic'].iloc[0])
        t1['mth'] = month_code
        macro_final = pd.concat([macro_final, t1], axis=0)
            
macro_final

,Mnemonic,M_FHOFHOPIPOQ.IUSA,M_FYLE.IUSA,M_FRT.IUSA,M_FYPSAVQ.IUSA,M_FC.IUSA,FYPCPICA_FEDB.IUSA,M_FGDP.IUSA,M_FYHHAVGQ.IUSA,M_FYFMMEDQ.IUSA,...,M_FETP.IUSA,M_FLTV.IUSA,M_FC$.IUSA,M_FEIAMGTTQ.IUSA,M_FXCPMGASUQ.IUSA,M_FRBBBQ.IUSA,M_FHOWNRQ.IUSA,M_FIFR_FEDS.IUSA,M_FIFR$_FEDS.IUSA,mth
223,1/31/2018,248.873161,10761.294,5181.42,7.1,13628.374,53049.86599,20073.62463,138178.1894,75120.61132,...,125.317,79.5,12775.504,2.6712,253.46,4.058025114,64.18299722,782.9992665,616.3053138,201801
224,2/28/2018,250.8678133,10780.022,5215.572,7.2,13668.318,53233.38442,20141.45839,138578.5727,75283.48447,...,125.672,79.3,12765.158,2.70475,256.9375,4.206673162,64.29943946,783.2181714,611.3780129,201802
225,3/31/2018,251.6692577,10808.394,5198.868,7.2,13735.355,53417.1033,20251.37579,139015.1391,75437.82054,...,125.85,79.2,12808.044,2.709,255.9775,4.374298511,64.41750914,784.8495621,608.5116733,201803
226,4/30/2018,252.3962837,10850.006,5220.732,7.2,13792.129,53607.57795,20361.7524,139415.5111,75597.31004,...,125.977,79.2,12863.202,2.8732,272.104,4.530713379,64.50872796,787.5053156,607.2579171,201804
227,5/31/2018,253.7272947,10884.605,5277.108,7.2,13860.171,53798.93234,20482.75073,139951.8411,75770.1069,...,126.238,78.9,12918.212,2.98675,290.12,4.625637301,64.52811155,789.9358207,606.4957157,201805
228,6/30/2018,254.8264638,10938.375,5250.324,7.4,13900.213,53991.49272,20566.09014,140544.0184,75971.59994,...,126.429,78.7,12946.163,2.9695,292.225,4.637646914,64.46222344,791.1338637,605.1493672,201806
229,7/31/2018,255.8970963,11000.556,5285.064,7.5,13952.254,54185.6046,20646.54348,141261.2749,76224.03195,...,126.562,79,12992.581,2.9278,288.464,4.6120216,64.37585739,790.9165229,603.0620194,201807
230,8/31/2018,257.1583704,11056.842,5278.512,7.6,14001.103,54384.72451,20705.33069,141810.1233,76556.76181,...,126.778,78.8,13035.279,2.91425,287.6625,4.617717228,64.3609134,789.3029976,600.2356039,201808
231,9/30/2018,257.5543485,11071.396,5272.152,7.7,14013.025,54582.57541,20709.96619,141969.5417,76971.1535,...,126.859,78,13031.547,2.91475,287.245,4.696259379,64.46533689,786.7324795,597.0563768,201809
232,10/31/2018,258.4575107,11075.242,5329.776,7.6,14096.453,54782.50649,20786.32137,142409.9521,77449.87931,...,127.064,78.4,13082.152,2.9426,289.058,4.809744127,64.60498768,784.3634066,594.1644085,201810


NEXT WE WILL STRATIFY SAMPLE THE CHARGED AND NON-CHARGED DATA

In [6]:
total_num = len(df_training)
charge_num = 0
for i in range(len(df_training)):
    if df_training["charge_off"][i] != 0:
        charge_num += 1


In [69]:
df_no_charge = df_training[df_training["charge_off"] == 0] 
df_no_charge_sample = df_no_charge.sample(n=15698)
df_charge = df_training[df_training["charge_off"] == 1]
df_clean = pd.concat([df_charge, df_no_charge_sample], axis=0)

In [110]:

x = pd.DataFrame()
final = pd.DataFrame()
a = 0 #
for i in df_clean['mth_code'].unique():
    left_block = df_clean[df_clean['mth_code'] == i]
    temp2 = macro_final[macro_final['mth'] == ((str)(i))]
    right_block = pd.concat([temp2]*len(left_block))
    
    # left_block = left_block.reset_index(drop=True)
    # right_block = right_block.reset_index(drop=True)
    
    x = pd.concat([left_block, right_block], axis=1, ignore_index=True)
    if a == 0:
        final = x
        a+=1
    else: 
        final = pd.concat([x, final], axis=0)
final
    

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [104]:
df_clean.columns.get_loc("charge_off")

22

In [106]:
data = []
for column in final.columns:
    data.append(pp.score(final, x=column, y=22))
pd.DataFrame(data).sort_values(by="ppscore", ascending=False).reset_index().head(n=20)


,index,x,y,ppscore,case,is_valid_score,metric,baseline_score,model_score,model
0,22,22,22,1.000000,predict_itself,True,None,0.0000,1.000000,None
1,21,21,22,1.000000,regression,True,mean absolute error,0.4966,0.000000,DecisionTreeRegressor()
2,27,27,22,0.998792,regression,True,mean absolute error,0.4966,0.000600,DecisionTreeRegressor()
3,50,50,22,0.988799,regression,True,mean absolute error,0.4966,0.005562,DecisionTreeRegressor()
4,26,26,22,0.980468,regression,True,mean absolute error,0.4966,0.009700,DecisionTreeRegressor()
5,28,28,22,0.871743,regression,True,mean absolute error,0.4966,0.063693,DecisionTreeRegressor()
6,35,35,22,0.841713,regression,True,mean absolute error,0.4966,0.078606,DecisionTreeRegressor()
7,36,36,22,0.786614,regression,True,mean absolute error,0.4966,0.105967,DecisionTreeRegressor()
8,23,23,22,0.774494,regression,True,mean absolute error,0.4966,0.111986,DecisionTreeRegressor()
9,18,18,22,0.688243,regression,True,mean absolute error,0.4966,0.154819,DecisionTreeRegressor()
